# Class Decorator

In [1]:
def savings(cls):
    cls.account_type = "Savings"
    return cls

In [2]:
def checking(cls):
    cls.account_type = "Checking"
    return cls

We can now use this decorator function to decorator the class.

In [3]:
class Account:
    pass

@savings
class Bank1Account:
    pass

@checking
class Bank2Account:
    pass

In [4]:
Bank1Account.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Account' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Account' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

In [5]:
Bank2Account.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank2Account' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank2Account' objects>,
              '__doc__': None,
              'account_type': 'Checking'})

We add the class attribute using the decorator function.Since we are using the code to modify the other code , Metaprogramming.

Of course, we could make even this simple example a little DRYer, by making a parameterized decorator:



# Parameter decorator

In [6]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [7]:
@account_type("Savings")
class Bank1Account:
    pass

@account_type("Checking")
class Bank2Account:
    pass

In [8]:
Bank2Account.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank2Account' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank2Account' objects>,
              '__doc__': None,
              'account_type': 'Checking'})

In [9]:
Bank1Account.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Account' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Account' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

In [10]:
Bank1Account.account_type ,Bank2Account.account_type

('Savings', 'Checking')

We can class decorator function to add the function to class also.

# Adding the function to class using the Decorator

In [11]:
def hello(cls):
    cls.hello = lambda self : f"{self} says hello!"
    return cls

In [12]:
@hello
class Person:
    def __init__(self,name):
        self.name = name
    def __str__(self):
        return str(self.name)

In [13]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [14]:
Person.hello

<function __main__.hello.<locals>.<lambda>(self)>

In [15]:
p = Person("python")

In [16]:
p.hello

<bound method hello.<locals>.<lambda> of <__main__.Person object at 0x000001D318ED6950>>

In [17]:
p.hello()

'python says hello!'

# Example the Logger each function in class

In [18]:
from functools import wraps

def func_logger(fn):
    @wraps(fn)
    def inner(*arg,**kwargs):
        result = fn(*arg,**kwargs)
        print(f"Log : {fn.__qualname__}({arg}, {kwargs}) = {result}")
        return result
    return inner

In [19]:
class Person:
    @func_logger
    def __init__(self,name):
        self.name = name
    @func_logger
    def greet(self):
        return f"Hello , my name is {self.name}"

In [20]:
p = Person("python")

Log : Person.__init__((<__main__.Person object at 0x000001D318F5A650>, 'python'), {}) = None


In [21]:
p.greet()

Log : Person.greet((<__main__.Person object at 0x000001D318F5A650>,), {}) = Hello , my name is python


'Hello , my name is python'

Using the function logger we can log teh each function .
but if class more function , we need to add this decorator to all function manually.
so we use the class decorator , that automatically do this for us.

In [22]:
for key,val in vars(Person).items():
    print(key,val)

__module__ __main__
__init__ <function Person.__init__ at 0x000001D318F46DE0>
greet <function Person.greet at 0x000001D318F46F20>
__dict__ <attribute '__dict__' of 'Person' objects>
__weakref__ <attribute '__weakref__' of 'Person' objects>
__doc__ None


In [23]:
def class_logger(cls):
    for name , obj in vars(cls).items():
        if callable(obj):
            print(f"decorating: {cls.__name__}  {name}")
            setattr(cls,name,func_logger(obj))
    return cls

In [24]:
@class_logger
class Person:
    def __init__(self,name):
        self.name = name

    def greet(self):
        return f"Hello , my name is {self.name}"

decorating: Person  __init__
decorating: Person  greet


In [25]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print("static.method")

    @classmethod
    def class_method(cls):
        print(f"class method for {cls}")


    def __init__(self, name):
        self.name = name


    def greet(self):
        return f"Hello , my name is {self.name}"

decorating: Person  static_method
decorating: Person  __init__
decorating: Person  greet


In [26]:
Person.static_method()

static.method
Log : Person.static_method((), {}) = None


In [27]:
p = Person("python")
p.greet()

Log : Person.__init__((<__main__.Person object at 0x000001D318F660D0>, 'python'), {}) = None
Log : Person.greet((<__main__.Person object at 0x000001D318F660D0>,), {}) = Hello , my name is python


'Hello , my name is python'

In [28]:
Person.class_method()

class method for <class '__main__.Person'>


class_method is not produce the logger , since class method are not callable , they non data descriptor.

In [29]:
callable(Person.__dict__["class_method"])

False

In [30]:
hasattr(Person.__dict__["class_method"],"__get__"),hasattr(Person.__dict__["class_method"],"__set__")

(True, False)

Which is probably a good thing, because our decorator is expecting to decorate a function, not a class!

This, by the way, is why when you decorate class methods using a function decorator in your classes, you should do so before you decorate it with the  @classmethod decorators:

In [31]:
class Person:
    @classmethod
    @func_logger
    def class_method(cls):
        pass

In [32]:
Person.class_method()

Log : Person.class_method((<class '__main__.Person'>,), {}) = None


Now the class method is logging. then why didn't logged when we are using the class decorator.
When we are using the classlogger , we are wrap only the callable function , but the class method is not callable ,they are non-data descriptor.

In [33]:
class Person:
    @func_logger
    @classmethod
    def class_method(cls):
        pass

In [34]:
try:
    Person.class_method()
except TypeError as ex:
    print(ex)

'classmethod' object is not callable


In [35]:
class Person:
    @classmethod
    def class_method(cls):
        pass

In [36]:
type(Person.__dict__["class_method"])

classmethod

In [37]:
type(classmethod)

type

so classmethod is the class ,which is nothing but the non data descriptor.
since it is descriptor, it store the original function `__func__` attribute.

In [38]:
Person.__dict__["class_method"].__func__

<function __main__.Person.class_method(cls)>

Now we wrap te original function and make the warp function as the classmethod. Problem solved.
so we have to distinguish the callable and classmethod in our class logger

In [39]:
def class_logger(cls):
    for name , obj in vars(cls).items():
        if callable(obj):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            setattr(cls,name,func_logger(obj))
        elif isinstance(obj,classmethod):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            class_method = classmethod(decorated_func)
            setattr(cls,name,class_method)
    return cls

In [40]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print("static.method")

    @classmethod
    def class_method(cls):
        print(f"class method for {cls}")

    def __init__(self, name):
        self.name = name

    def greet(self):
        return f"Hello , my name is {self.name}"

decorating <class 'staticmethod'>: Person  static_method
decorating <class 'classmethod'>: Person  class_method
decorating <class 'function'>: Person  __init__
decorating <class 'function'>: Person  greet


In [41]:
type(Person.static_method)

function

# what about the property

In [42]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print("static.method")

    @classmethod
    def class_method(cls):
        print(f"class method for {cls}")

    def __init__(self, name):
        self._name = name

    def greet(self):
        return f"Hello , my name is {self.name}"

    @property
    def name(self):
        return self._name

decorating <class 'staticmethod'>: Person  static_method
decorating <class 'classmethod'>: Person  class_method
decorating <class 'function'>: Person  __init__
decorating <class 'function'>: Person  greet


property didn't get decorated. why?
same reason , property is not callable and it data descriptor

In [43]:
type(Person.__dict__["name"])

property

In [44]:
hasattr(Person.__dict__["name"],"__get__"),hasattr(Person.__dict__["name"],"__set__")

(True, True)

In [45]:
isinstance(classmethod,type)

True

In [46]:
isinstance(property,type)

True

In [47]:
isinstance(classmethod,property)

False

In [48]:
prop = Person.__dict__["name"]

In [49]:
prop.fget

<function __main__.Person.name(self)>

In [50]:
prop.fset,prop.fdel

(None, None)

Hmm, so maybe we can decorate the `fget`, `fset`, and `fdel` functions of the property (if they are not None).

We can't just replace the functions, because `fget`, `fset` and `fdel` are actually read-only properties themselves, that return the original functions. But we could create a new property based off thge original one, substituting our decorated getter, setter and deleter.

Recall that the `getter(), setter() and deleter()` methods are methods that will create a copy of the original property, but substitute the fget, fset and `fdel` methods (that's how these are used as decorators).

In [51]:
def class_logger(cls):
    for name , obj in vars(cls).items():
        if callable(obj):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            setattr(cls,name,func_logger(obj))
        elif isinstance(obj,classmethod):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            class_method = classmethod(decorated_func)
            setattr(cls,name,class_method)
        elif isinstance(obj,property):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls,name,obj)
    return cls

In [52]:
@class_logger
class Person:
    @staticmethod
    def static_method():
        print("static.method")

    @classmethod
    def class_method(cls):
        print(f"class method for {cls}")

    def __init__(self, name):
        self._name = name

    def greet(self):
        return f"Hello , my name is {self.name}"

    @property
    def name(self):
        return self._name

decorating <class 'staticmethod'>: Person  static_method
decorating <class 'classmethod'>: Person  class_method
decorating <class 'function'>: Person  __init__
decorating <class 'function'>: Person  greet
decorating <class 'property'>: Person  name


In [53]:
p = Person("python")

Log : Person.__init__((<__main__.Person object at 0x000001D318F7AF90>, 'python'), {}) = None


In [54]:
p.name

Log : Person.name((<__main__.Person object at 0x000001D318F7AF90>,), {}) = python


'python'

In [55]:
try:
    p.name = "change"
except AttributeError as ex:
    print(ex)

property 'name' of 'Person' object has no setter


In [56]:
try:
    del p.name
except AttributeError as ex:
    print(ex)

property 'name' of 'Person' object has no deleter


Still, this is not perfect... :(

We can still run into trouble because not every callable is a function that can be decorated:

In [57]:
@class_logger
class Person:
    class Other:
        def __call__(self, *args, **kwargs):
            pass
    other = Other()

decorating <class 'type'>: Person  Other
decorating <class '__main__.Person.Other'>: Person  other


In [58]:
Person.Other

<function __main__.Person.Other()>

In [59]:
Person.other

<function __main__.func_logger.<locals>.inner(*args, **kwargs)>

So, as you see it decorated both the class Other (since classes are callables), and it decorated other since we made instances of Other callable too.

How does that work with the logger though:

In [60]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def cls_method(cls):
        pass

    def inst_method(self):
        pass

    @property
    def name(self):
        pass

    def __add__(self, other):
        pass

    class Other:
        def __call__(self):
            pass

    other = Other()


decorating <class 'staticmethod'>: MyClass  static_method
decorating <class 'classmethod'>: MyClass  cls_method
decorating <class 'function'>: MyClass  inst_method
decorating <class 'property'>: MyClass  name
decorating <class 'function'>: MyClass  __add__
decorating <class 'type'>: MyClass  Other
decorating <class '__main__.MyClass.Other'>: MyClass  other


In [61]:
import inspect

In [62]:
keys = ('static_method', 'cls_method', 'inst_method', 'name', '__add__', 'Other', 'other')
inspect_funcs = ('isroutine', 'ismethod', 'isfunction', 'isbuiltin', 'ismethoddescriptor','isdatadescriptor')

In [63]:
max_header_length = max(len(key) for key in keys)
max_fname_length = max(len(func) for func in inspect_funcs)
print(format('', f'{max_fname_length}s'), '\t'.join(format(key, f'{max_header_length}s') for key in keys))
for inspect_func in inspect_funcs:
    fn = getattr(inspect, inspect_func)
    inspect_results = (format(str(fn(MyClass.__dict__[key])), f'{max_header_length}s') for key in keys)
    print(format(inspect_func, f'{max_fname_length}s'), '\t'.join(inspect_results))

                   static_method	cls_method   	inst_method  	name         	__add__      	Other        	other        
isroutine          True         	True         	True         	False        	True         	True         	True         
ismethod           False        	False        	False        	False        	False        	False        	False        
isfunction         True         	False        	True         	False        	True         	True         	True         
isbuiltin          False        	False        	False        	False        	False        	False        	False        
ismethoddescriptor False        	True         	False        	False        	False        	False        	False        
isdatadescriptor   False        	False        	False        	True         	False        	False        	False        


In [64]:
[method for method in dir(inspect) if method.startswith("is")]

['isabstract',
 'isasyncgen',
 'isasyncgenfunction',
 'isawaitable',
 'isbuiltin',
 'isclass',
 'iscode',
 'iscoroutine',
 'iscoroutinefunction',
 'isdatadescriptor',
 'isframe',
 'isfunction',
 'isgenerator',
 'isgeneratorfunction',
 'isgetsetdescriptor',
 'iskeyword',
 'ismemberdescriptor',
 'ismethod',
 'ismethoddescriptor',
 'ismethodwrapper',
 'ismodule',
 'isroutine',
 'istraceback']

In [65]:
def class_logger(cls):
    for name , obj in vars(cls).items():
        if isinstance(obj,classmethod):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            original_func = obj.__func__
            decorated_func = func_logger(original_func)
            class_method = classmethod(decorated_func)
            setattr(cls,name,class_method)
        elif isinstance(obj,property):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            if obj.fget:
                obj = obj.getter(func_logger(obj.fget))
            if obj.fset:
                obj = obj.setter(func_logger(obj.fset))
            if obj.fdel:
                obj = obj.deleter(func_logger(obj.fdel))
            setattr(cls,name,obj)
        elif inspect.isroutine(obj):
            print(f"decorating {type(obj)}: {cls.__name__}  {name}")
            setattr(cls,name,func_logger(obj))
    return cls

In [66]:
@class_logger
class MyClass:
    @staticmethod
    def static_method():
        pass

    @classmethod
    def cls_method(cls):
        pass

    def inst_method(self):
        pass

    @property
    def name(self):
        pass

    def __add__(self, other):
        pass

    class Other:
        def __call__(self):
            pass

    other = Other()


decorating <class 'staticmethod'>: MyClass  static_method
decorating <class 'classmethod'>: MyClass  cls_method
decorating <class 'function'>: MyClass  inst_method
decorating <class 'property'>: MyClass  name
decorating <class 'function'>: MyClass  __add__
